# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f68e4ca1b20>
├── 'a' --> tensor([[ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162]])
└── 'x' --> <FastTreeValue 0x7f68e4ca1f40>
    └── 'c' --> tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                        [ 0.2126,  0.3855,  0.7497,  0.3247],
                        [-0.4739,  0.4981,  0.6090, -0.5094]])

In [4]:
t.a

tensor([[ 0.1980,  1.3459, -1.1448],
        [-1.7813,  0.6081, -0.4162]])

In [5]:
%timeit t.a

75.5 ns ± 0.761 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f68e4ca1b20>
├── 'a' --> tensor([[ 0.2912, -0.4848, -1.7786],
│                   [ 1.7867, -0.4711,  0.4483]])
└── 'x' --> <FastTreeValue 0x7f68e4ca1f40>
    └── 'c' --> tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                        [ 0.2126,  0.3855,  0.7497,  0.3247],
                        [-0.4739,  0.4981,  0.6090, -0.5094]])

In [7]:
%timeit t.a = new_value

76.4 ns ± 0.612 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162]]),
    x: Batch(
           c: tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                      [ 0.2126,  0.3855,  0.7497,  0.3247],
                      [-0.4739,  0.4981,  0.6090, -0.5094]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1980,  1.3459, -1.1448],
        [-1.7813,  0.6081, -0.4162]])

In [11]:
%timeit b.a

68.7 ns ± 0.344 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6717, -1.0541, -0.0569],
               [ 1.4706,  0.2650,  0.6625]]),
    x: Batch(
           c: tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                      [ 0.2126,  0.3855,  0.7497,  0.3247],
                      [-0.4739,  0.4981,  0.6090, -0.5094]]),
       ),
)

In [13]:
%timeit b.a = new_value

664 ns ± 7.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.3 µs ± 249 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

275 µs ± 8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

267 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f682df5e700>
├── 'a' --> tensor([[[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]],
│           
│                   [[ 0.1980,  1.3459, -1.1448],
│                    [-1.7813,  0.6081, -0.4162]]])
└── 'x' --> <FastTreeValue 0x7f682df5e2b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.5 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6834bcf550>
├── 'a' --> tensor([[ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162],
│                   [ 0.1980,  1.3459, -1.1448],
│                   [-1.7813,  0.6081, -0.4162]])
└── 'x' --> <FastTreeValue 0x7f6834bcf3d0>
    └── 'c' --> tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                        [ 0.2126,  0.3855,  0.7497,  0.3247],
                 

In [23]:
%timeit t_cat(trees)

44.3 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

83.3 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]],
       
               [[ 0.1980,  1.3459, -1.1448],
                [-1.7813,  0.6081, -0.4162]]]),
    x: Batch(
           c: tensor([[[ 1.0442, -1.3419, -1.3608,  1.8621],
                       [ 0.2126,  0.3855,  0.7497,  0.3247],
                       [-0.4739,  0.4981,  0.6090, -0.5094]],
         

In [26]:
%timeit Batch.stack(batches)

111 µs ± 788 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162],
               [ 0.1980,  1.3459, -1.1448],
               [-1.7813,  0.6081, -0.4162]]),
    x: Batch(
           c: tensor([[ 1.0442, -1.3419, -1.3608,  1.8621],
                      [ 0.2126,  0.3855,  0.7497,  0.3247],
                      [-0.4739,  0.4981,  0.6090, -0.5094],
                      [ 1.0442, -1.3419, -1.3608,  1.8621],
                      [ 0.2126,  

In [28]:
%timeit Batch.cat(batches)

200 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

553 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
